In [2]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

 Se hará una red neuronal que reciba como input el coeficiente de absorción y como output tengamos radio, fracción volumen y espesor de la película

# Juntando todos los archivos en uno solo

In [3]:
files = [file for file in os.listdir('./parametros')]

param = pd.DataFrame()
for file in files:
  df = pd.read_csv('./parametros/'+file)
  param = pd.concat([param, df], axis= 0)

In [4]:
param

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
0,301,0.016685,1,10,100
1,302,0.016167,1,10,100
2,303,0.015703,1,10,100
3,304,0.015290,1,10,100
4,305,0.014916,1,10,100
...,...,...,...,...,...
19195,896,0.002255,1,9,120
19196,897,0.002257,1,9,120
19197,898,0.002258,1,9,120
19198,899,0.002260,1,9,120


# Revolviendo todos los datos

Esto es para el entrenamiento óptimo de la red y poderle dar una mayor gama de datos

In [5]:
param = param.sample(frac=1, random_state=42).reset_index(drop=True)


In [6]:
param

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
0,753,0.001533,1,23,120
1,310,0.014692,3,10,120
2,625,0.002815,1,10,85
3,604,0.000572,1,9,120
4,373,0.003104,1,21,120
...,...,...,...,...,...
38395,566,0.000711,1,10,70
38396,785,0.000418,12,10,120
38397,659,0.000772,1,9,120
38398,561,0.005686,1,10,105


In [7]:
from sklearn.model_selection import train_test_split


# Separando nuestra base de datos en 2, una que servirá para entrenar el modelo y otra para hacerle un test

In [8]:
train_df, test_df = train_test_split(param, test_size=0.2, random_state=42)

test será el 20% de los datos mientras train el 80%

In [9]:
test_df

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
27452,492,0.006226,5,10,120
15009,831,0.001993,1,26,120
8124,700,0.000512,1,26,120
38103,748,0.000418,11,10,120
25872,432,0.002877,1,10,65
...,...,...,...,...,...
3746,820,0.002003,7,10,120
18195,422,0.000481,1,7,120
37286,551,0.033743,4,10,120
15361,703,0.000239,1,29,120


In [10]:
train_df

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
36772,825,0.001564,11,10,120
22714,538,0.004552,1,10,65
3031,442,0.000189,1,10,120
9632,391,0.000021,1,10,115
29786,870,0.002197,1,11,120
...,...,...,...,...,...
6265,760,0.002772,1,10,100
11284,623,0.001487,1,32,120
38158,836,0.001522,1,10,135
860,324,0.008735,1,6,120


# Prueba 1

In [11]:
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
data = train_df
input_data = data.drop(['Fracvol', 'Longitud de onda', 'Radio (nm)', 'Espesor de la película (nm)'],axis=1).values


In [13]:

fracvol = data.drop(['Índice de absorción', 'Longitud de onda', 'Radio (nm)', 'Espesor de la película (nm)'],axis=1).values
radio = data.drop(['Índice de absorción', 'Longitud de onda', 'Fracvol', 'Espesor de la película (nm)'],axis=1).values
esp_pel = data.drop(['Índice de absorción', 'Longitud de onda', 'Fracvol', 'Radio (nm)'],axis=1).values

In [14]:
inputs = keras.Input(shape=(3,), name='entrada')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
output1_layer = layers.Dense(1, name='fracvol')(x)
output2_layer = layers.Dense(1, name='radio')(x)
output3_layer = layers.Dense(1, name='esp_pel')(x)

In [15]:
model = keras.Model(inputs=inputs, outputs=[output1_layer, output2_layer, output3_layer])

# Compilar el modelo
model.compile(optimizer='adam',
              loss={'output1': 'binary_crossentropy', 'output2': 'binary_crossentropy', 'output3': 'binary_crossentropy'},
              metrics=['accuracy'])

In [16]:
# Entrenar el modelo
model.fit(input_data, {'fracvol': fracvol, 'radio': radio, 'esp_pel': esp_pel}, epochs=100, batch_size=1)


Epoch 1/100


ValueError: in user code:

    File "C:\Users\PcR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\PcR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\PcR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\PcR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1023, in train_step
   

No sirvió tratando de hacer fit el modelo

# Prueba 2

In [17]:
# Crear un DataFrame de entrada y salida
data = param

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# Obtener los datos de entrada y salida para el conjunto de entrenamiento y prueba
train_input_data = train_data['Índice de absorción'].values
train_output_data = train_data[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']].values
test_input_data = test_data['Índice de absorción']
test_output_data = test_data[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]



In [18]:
# Crear el modelo
inputs = keras.Input(shape=(1,), name='input')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)


In [19]:
output1_layer = layers.Dense(1, name='Fracvol')(x)
output3_layer = layers.Dense(1, name='espesor_pelicula_nm')(x)
output2_layer = layers.Dense(1, name='Radio_nm')(x)


In [20]:
model = keras.Model(inputs=inputs, outputs=[output1_layer, output2_layer, output3_layer])

# Compilar el modelo
model.compile(optimizer='adam',
              loss={'Fracvol': 'binary_crossentropy', 'Radio_nm': 'binary_crossentropy', 'espesor_pelicula_nm': 'binary_crossentropy'},metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_input_data, train_output_data, epochs=2, batch_size=1)

Epoch 1/2
30720/30720 [==============================] - 29s 909us/step - loss: -1952.5455 - Fracvol_loss: -650.4683 - Radio_nm_loss: -650.7025 - espesor_pelicula_nm_loss: -651.3922 - Fracvol_accuracy: 0.2532 - Radio_nm_accuracy: 0.2532 - espesor_pelicula_nm_accuracy: 0.2535
Epoch 2/2
30720/30720 [==============================] - 28s 900us/step - loss: -1974.8123 - Fracvol_loss: -658.2731 - Radio_nm_loss: -658.2731 - espesor_pelicula_nm_loss: -658.2731 - Fracvol_accuracy: 0.2554 - Radio_nm_accuracy: 0.2554 - espesor_pelicula_nm_accuracy: 0.2554


In [21]:
arr = train_data['Índice de absorción'].to_numpy()

In [22]:
predictions = model.predict(arr)
predictions

960/960 [==============================] - 1s 689us/step


[array([[1.2751088],
        [1.2610755],
        [1.2561675],
        ...,
        [1.258758 ],
        [1.2622546],
        [1.2558494]], dtype=float32),
 array([[1.4972441],
        [1.4780445],
        [1.4713296],
        ...,
        [1.4748737],
        [1.4796579],
        [1.4708942]], dtype=float32),
 array([[1.7655803],
        [1.7428479],
        [1.7348975],
        ...,
        [1.7390937],
        [1.7447582],
        [1.7343822]], dtype=float32)]

In [23]:
posit = []
for fila in predictions:
    for num in fila:
        if num > 0:
            posit.append(num)

    

In [ ]:
posit

[array([1.4422398], dtype=float32),
 array([1.4206411], dtype=float32),
 array([1.4130871], dtype=float32),
 array([1.4126086], dtype=float32),
 array([1.425154], dtype=float32),
 array([1.4216166], dtype=float32),
 array([1.4149988], dtype=float32),
 array([1.4223944], dtype=float32),
 array([1.4144057], dtype=float32),
 array([1.465299], dtype=float32),
 array([1.4294347], dtype=float32),
 array([1.4184968], dtype=float32),
 array([1.4173148], dtype=float32),
 array([1.4228536], dtype=float32),
 array([1.4187722], dtype=float32),
 array([1.4282593], dtype=float32),
 array([1.4115937], dtype=float32),
 array([1.4152589], dtype=float32),
 array([1.4185393], dtype=float32),
 array([1.416246], dtype=float32),
 array([1.4120572], dtype=float32),
 array([1.4121276], dtype=float32),
 array([1.4147711], dtype=float32),
 array([1.4130442], dtype=float32),
 array([1.4826787], dtype=float32),
 array([1.4213822], dtype=float32),
 array([1.4318736], dtype=float32),
 array([1.4119595], dtype=float

Nos da números pero no tienen significado físico, como negativos

# Prueba 3

In [24]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Cargar DataFrame
dataframe = train_df
# Separar datos de entrada y salida
input_data = dataframe['Índice de absorción']
output_data = dataframe[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

# Crear modelo de red neuronal
model = Sequential()

# Agregar capa de entrada
model.add(Dense(units=8, input_dim=1, activation='relu'))

# Agregar capas ocultas
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))

# Agregar capa de salida
model.add(Dense(units=3, activation='sigmoid'))

# Compilar modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar modelo
model.fit(input_data, output_data, epochs=10, batch_size=32)

Epoch 1/10
960/960 [==============================] - 1s 728us/step - loss: -64404.7109 - accuracy: 1.0000
Epoch 2/10
960/960 [==============================] - 1s 713us/step - loss: -1652139.3750 - accuracy: 1.0000
Epoch 3/10
960/960 [==============================] - 1s 686us/step - loss: -8920462.0000 - accuracy: 1.0000
Epoch 4/10
960/960 [==============================] - 1s 679us/step - loss: -26572992.0000 - accuracy: 1.0000
Epoch 5/10
960/960 [==============================] - 1s 676us/step - loss: -59061692.0000 - accuracy: 1.0000
Epoch 6/10
960/960 [==============================] - 1s 677us/step - loss: -110856704.0000 - accuracy: 1.0000
Epoch 7/10
960/960 [==============================] - 1s 680us/step - loss: -186611296.0000 - accuracy: 1.0000
Epoch 8/10
960/960 [==============================] - 1s 678us/step - loss: -291276768.0000 - accuracy: 1.0000
Epoch 9/10
960/960 [==============================] - 1s 677us/step - loss: -430176544.0000 - accuracy: 1.0000
Epoch 10/10

In [ ]:
arr = train_data['Índice de absorción'].to_numpy()

Haciendo predicciones

In [ ]:
predictions = model.predict(arr)
predictions

960/960 [==============================] - 1s 523us/step


array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       ...,
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

Quitando los unos de las predicciones

In [ ]:
indices = np.where(predictions != 1)
indices

(array([], dtype=int64), array([], dtype=int64))

Nos da puros unos como resultados

# Prueba 4

In [25]:
# Cargar DataFrame
df = train_df
# Separar datos de entrada y salida
X = df['Índice de absorción']
Y = df[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [26]:
# Convertimos los dataframes en numpy arrays
X = np.array(X).reshape(-1, 1)
y = np.array(Y)


In [27]:
# Creamos el modelo de la red neuronal
model = Sequential()
model.add(Dense(10, input_dim=1, activation='relu'))
model.add(Dense(3, activation='linear'))

In [28]:
model.compile(loss='mean_squared_error', optimizer='adam')


In [29]:
model.fit(X, y, epochs=100)


Epoch 1/100
960/960 [==============================] - 1s 603us/step - loss: 4238.4902
Epoch 2/100
960/960 [==============================] - 1s 677us/step - loss: 3015.2312
Epoch 3/100
960/960 [==============================] - 1s 642us/step - loss: 1565.4862
Epoch 4/100
960/960 [==============================] - 1s 617us/step - loss: 485.8235
Epoch 5/100
960/960 [==============================] - 1s 604us/step - loss: 141.9248
Epoch 6/100
960/960 [==============================] - 1s 600us/step - loss: 105.5447
Epoch 7/100
960/960 [==============================] - 1s 608us/step - loss: 104.7770
Epoch 8/100
960/960 [==============================] - 1s 598us/step - loss: 104.7678
Epoch 9/100
960/960 [==============================] - 1s 635us/step - loss: 104.7502
Epoch 10/100
960/960 [==============================] - 1s 649us/step - loss: 104.7025
Epoch 11/100
960/960 [==============================] - 1s 632us/step - loss: 104.7045
Epoch 12/100
960/960 [===========================

In [30]:
# Predecimos los valores de salida para un nuevo dato de entrada
X_new = np.array([6]).reshape(-1, 1)
X_new
arr = train_data['Índice de absorción'].to_numpy()


In [31]:
y_pred = model.predict(arr)
print(y_pred)


960/960 [==============================] - 1s 526us/step
[[  3.1947927  15.098988  114.46636  ]
 [  2.669147   15.568951  114.35653  ]
 [  2.4853077  15.7333145 114.31811  ]
 ...
 [  2.5823388  15.646563  114.33839  ]
 [  2.7133174  15.529459  114.36575  ]
 [  2.4733891  15.743971  114.31562  ]]


In [33]:
predicciones = pd.DataFrame(y_pred)
predicciones

,0,1,2
0,3.194793,15.098988,114.466362
1,2.669147,15.568951,114.356529
2,2.485308,15.733315,114.318108
3,2.473661,15.743729,114.315681
4,2.778978,15.470755,114.379471
...,...,...,...
30715,4.210559,14.190821,114.678604
30716,2.681565,15.557848,114.359123
30717,2.582339,15.646563,114.338387
30718,2.713317,15.529459,114.365753


# Renombrando las columnas

In [34]:
predicciones.columns = ['Fracción Volumen', 'Radio (nm)', 'Espesor de la película (nm)']

# Redondeando los valores

In [35]:
predicciones = predicciones.round()

In [36]:
predicciones = predicciones.join(train_data['Índice de absorción'])

In [37]:
predicciones

,Fracción Volumen,Radio (nm),Espesor de la película (nm),Índice de absorción
0,3.0,15.0,114.0,0.001533
1,3.0,16.0,114.0,0.014692
2,2.0,16.0,114.0,0.002815
3,2.0,16.0,114.0,0.000572
4,3.0,15.0,114.0,0.003104
...,...,...,...,...
30715,4.0,14.0,115.0,0.000163
30716,3.0,16.0,114.0,0.003398
30717,3.0,16.0,114.0,0.000499
30718,3.0,16.0,114.0,0.001336


In [ ]:
predicciones.to_csv('predicciones100.csv', index=False)


In [38]:
pred10 = pd.read_csv("predicciones10.csv")
pred100 = pd.read_csv("predicciones100.csv")


In [48]:
dif = pred100-pred10

In [49]:
df4 = pd.DataFrame(dif.values[dif.values != 0])


In [51]:
dif

,Fracción Volumen,Radio (nm),Espesor de la película (nm),Índice de absorción
0,0.0,0.0,5.0,0.0
1,0.0,0.0,6.0,0.0
2,0.0,0.0,6.0,0.0
3,0.0,0.0,6.0,0.0
4,0.0,0.0,6.0,0.0
...,...,...,...,...
30715,0.0,0.0,4.0,0.0
30716,0.0,0.0,6.0,0.0
30717,0.0,0.0,6.0,0.0
30718,0.0,0.0,6.0,0.0


In [55]:
df_filtrado = dif.loc[(dif != 0).any(axis=1)]


In [56]:
df_filtrado

,Fracción Volumen,Radio (nm),Espesor de la película (nm),Índice de absorción
0,0.0,0.0,5.0,0.0
1,0.0,0.0,6.0,0.0
2,0.0,0.0,6.0,0.0
3,0.0,0.0,6.0,0.0
4,0.0,0.0,6.0,0.0
...,...,...,...,...
30715,0.0,0.0,4.0,0.0
30716,0.0,0.0,6.0,0.0
30717,0.0,0.0,6.0,0.0
30718,0.0,0.0,6.0,0.0
